## Imports

In [1]:
import sys
sys.path.append('python/')
import numpy as np
import lmfit as lf
import matplotlib.pyplot as plt
import scipy.optimize as opt
import dataPython as dp
from scipy.integrate import quad
import scipy.optimize as so
import scipy.special as ss

In [2]:
## Measured points
#Measured points from Noordermeer's paper:

#data = dp.getXYdata_wXYerr('data/NGC5533-rot-data_fmt.txt')
#r_dat = np.asarray(data['xx'])
#v_dat = np.asarray(data['yy'])
#v_err0 = np.asarray(data['ex'])
#v_err1 = np.asarray(data['ey'])

## Bulge

In [3]:
%run bulge.ipynb



ERROR:root:File `'bulge.ipynb.py'` not found.


In [4]:
print(len(r_b_nordold))

##Noordermeer's Bulge curve
data_bulge_nord = dp.getXYdata('data/NGC5533-bulge_fmt.txt')

#convert to numpy arrays
r_b_nordold = np.asarray(data_bulge_nord['xx'])
v_b_nordold = np.asarray(data_bulge_nord['yy'])

#getting data down to 200 points to be comparable to function

#rdata
i=0
r_b_nord = np.zeros((int(200)))
while i<len(r_b_nord):
    r_b_nord[i]=r_b_nordold[4*i]
    i +=1

#ydata
i=0
v_b_nord = np.zeros((int(200)))
while i<len(v_b_nord):
    v_b_nord[i]=v_b_nordold[4*i]
    i +=1
    




NameError: name 'r_b_nordold' is not defined

In [ ]:
#function
def v(r,b):
    return b*vb

#(troubleshooting)seeing if values are decreasing
i = 0
while i < len(r_b_nord)-1:
    print(r_b_nord[i+1] - r_b_nord[i])
    i += 1

In [ ]:
#LMFIT FITTING
#Setup
#weighdata = 1/sigdata                       #We will need weights for lmfit. This tells us how much to account for a single data point in the fit.
l_mod = lf.Model(v)                         #Tell lmfit that we want to model the function f
params = l_mod.make_params(b=1)   #Give lmfit our initial guesses - again, the true function

#Do fit
l_fit = l_mod.fit(v_b_nord, params, r=r_b_nord, nan_policy='omit') #Here is where the weights we set at the beginning come in.

#Define Stuff


l_dict = l_fit.best_values #l_fit has a lot of output. We want to use the final result.
l_b = l_dict['b'] 


import scipy.interpolate as inter

#vbspline = inter.InterpolatedUnivariateSpline(r_b_nord, vb, k=3)    #turns array vb into an approximate function

#Create array to plot
l_curve = lambda r, l_b: l_b*vb
#l_curve = np.sqrt(l_G*l_M/r_bh_nord)  #KITTYCHANGE again, just equation only now with these second array things

l_fit #Display information about the fit

In [ ]:
#Plotting

fig = plt.figure(figsize=(9.0,8.0))                #size of the plot
plt.plot(r_b_nord,l_curve(r_b_nord,l_b),linestyle='-',label='LmFit')

plt.errorbar(r_b_nord,v_b_nord,fmt='r',label='Data',linestyle='--') #Plot points (fmt='*o') with error bars
#plt.plot(r_b_nord,ycurve,label='Function')                      #Accepting default style gives us a solid line
#plt.plot(r_b_nord,s_curve,linestyle='--',label='SciPy')         #Plot the scipy curve fit with a dashed line

plt.legend()           #Tells our plot to show a legend
plt.show()             #Depending how your environment is set up, this line may not be necessary, but it won't break anything.